In [10]:
import pandas as pd
import sidetable
import regex as re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer

Vamos a trabajar con el csv del pair de Limpieza III e intentar eliminar los valores nulos de nuestras columnas. En la lección hemos aprendido varios métodos de skelarn intentemos aplicarlos todos. 
- Es el momento de eliminar los nulos:


In [11]:
df = pd.read_csv("datos/attacks_pandas_10b.csv", index_col= 0)

Reemplazad los valores nulos de la columna age por la media de la edad usando el método SimpleImputer.


In [12]:
imputer = SimpleImputer(strategy="mean", missing_values=np.nan,)

In [13]:
imputer = imputer.fit(df[["age"]])

In [14]:
df["age"]= imputer.transform(df[["age"]])

In [15]:
df.sample()

,case_number,year,type,country,area,location,age,species_,date,mes,fatal_(y/n)_limpio,sex,country_
3321,2002.09.21.b,NaN,Unprovoked,Unknown,NaN,NaN,25.0,Unspecified,21-Sep-2002,Sep,N,F,NaN


Reemplazad los valores nulos de la columna sex por la moda, usando el método SimpleImputer.
💡 Pista 💡 La moda en este tipo de aproximación se indica como most_frequent.


In [16]:
# Tenemos dos columnas de sex, la que ya habíamos limpiado anteriormente en Pandas sin nulos, y la que no estaba limpia "sex_". Usamos esa.

df.describe(include="object")

,case_number,type,country,area,location,species_,date,mes,fatal_(y/n)_limpio,sex,country_
count,7788,7789,7789,1626,1621,7789,7789,5394,7789,7789,1662
unique,6287,6,96,236,1310,6,5432,22,3,2,95
top,1959.11.22,Unprovoked,Unknown,Florida,"New Smyrna Beach, Volusia County",Unspecified,Unknown,Jul,N,M,usa
freq,2,7549,6127,346,73,7101,1503,620,4292,4519,741


In [17]:
imputer2 = SimpleImputer(strategy="most_frequent", missing_values=np.nan,)

In [18]:
imputer2 = imputer2.fit(df[["sex"]])

In [21]:
df["sex"]= imputer2.transform(df[["sex"]])

In [24]:
df["sex"].isnull().sum()

0

In [25]:
df.describe(include="object")

,case_number,type,country,area,location,species_,date,mes,fatal_(y/n)_limpio,sex,country_
count,7788,7789,7789,1626,1621,7789,7789,5394,7789,7789,1662
unique,6287,6,96,236,1310,6,5432,22,3,2,95
top,1959.11.22,Unprovoked,Unknown,Florida,"New Smyrna Beach, Volusia County",Unspecified,Unknown,Jul,N,M,usa
freq,2,7549,6127,346,73,7101,1503,620,4292,4519,741


Reemplazad los valores nulos de la columna type por el valor más frecuente (la moda) con el método SimpleImputer.


In [27]:
# Igual que en el pair anterior, como interpretamos que para probar las hipótesis no necesitábamos de la columna type la borramos en limpieza 1, por tanto haremos
# el ejercicio en la columna "injury"

imputer3 = SimpleImputer(strategy="most_frequent", missing_values=np.nan,)
imputer3 = imputer3.fit(df[["type"]])
df["type"]= imputer3.transform(df[["type"]])


In [29]:
df["type"].isnull().sum()

0

In [30]:
df.describe(include="object")

,case_number,type,country,area,location,species_,date,mes,fatal_(y/n)_limpio,sex,country_
count,7788,7789,7789,1626,1621,7789,7789,5394,7789,7789,1662
unique,6287,6,96,236,1310,6,5432,22,3,2,95
top,1959.11.22,Unprovoked,Unknown,Florida,"New Smyrna Beach, Volusia County",Unspecified,Unknown,Jul,N,M,usa
freq,2,7549,6127,346,73,7101,1503,620,4292,4519,741


Utilizad el método KNN Imputer para reemplazar todos los valores nulos de las columnas numéricas.


In [31]:
df_copia1 = df.copy()

In [32]:
df_numericas= df_copia1.select_dtypes(include=np.number)

In [34]:
imputerKNN = KNNImputer(n_neighbors=5)
imputerKNN.fit(df_numericas)

KNNImputer()

In [35]:
numericas_KNN = imputerKNN.transform(df_numericas)

In [36]:
df_KNN_imputer = pd.DataFrame(numericas_KNN, columns= df_numericas.columns)

In [37]:
df_KNN_imputer.head()

,year,age
0,2018.0,57.0
1,2018.0,18.0
2,2018.0,15.0
3,2018.0,32.0
4,2018.0,21.0


In [38]:
columnas_df_KNN_imputer = df_KNN_imputer.columns

In [39]:
df_copia1.drop(columnas_df_KNN_imputer,axis=1,inplace=True)

In [40]:
df_copia1[columnas_df_KNN_imputer] = df_KNN_imputer

In [41]:
df_copia1["year"].isnull().sum()

0

Utilizad el método Iterative Imputer para reemplazar todos los valores nulos de las columnas numéricas.


In [42]:
df_copia2 = df.copy()

In [43]:
numericas_iterative = df_copia2.select_dtypes(include= np.number)

In [44]:
i_imputer = IterativeImputer(n_nearest_features= None, imputation_order= "ascending")

In [45]:
i_imputer.fit(numericas_iterative)

IterativeImputer()

In [46]:
i_imputer.transform(numericas_iterative)

array([[2018.        ,   57.        ],
       [2018.        ,   18.        ],
       [2018.        ,   15.        ],
       ...,
       [1991.58334433,   25.        ],
       [1991.58334433,   25.        ],
       [1991.58334433,   25.        ]])

In [47]:
df_numericas_trans = pd.DataFrame(i_imputer.transform(numericas_iterative), columns= numericas_iterative.columns)

In [48]:
df_numericas_trans.head()

,year,age
0,2018.0,57.0
1,2018.0,18.0
2,2018.0,15.0
3,2018.0,32.0
4,2018.0,21.0


In [49]:
df_numericas_trans.isnull().sum()

year    0
age     0
dtype: int64

In [50]:
nombres = df_numericas_trans.columns

In [51]:
df_copia2.drop(nombres,axis=1,inplace=True)

In [52]:
df_copia2[nombres] = df_numericas_trans

In [53]:
df_copia2.isnull().sum()

case_number              1
type                     0
country                  0
area                  6163
location              6168
species_                 0
date                     0
mes                   2395
fatal_(y/n)_limpio       0
sex                      0
country_              6127
year                     0
age                      0
dtype: int64

¿Podríais explicar qué diferencia hay entre estos dos últimos métodos?


Por lo que podemos observar el Iterative Imputer se basa en los datos cercanos para crear una predicción que se ajuste a lo que observa de los datos preexistentes, y el KNN busca como patrones en los datos para agruparlos y meter el nan en el grupo al que más se asemeje.

Aparte el Iterative Imputer podemos pasarle más parámetros para controlar el resultado (que sea la moda, la medias, mediana...) y el orden de sustitución.

Guardad el csv para seguir trabajando con el en los siguientes ejercicios de pair

In [54]:
# Como consideramos que el KNN y el II  han hehco un trabajo similar guardamos solo el último de los df donde hemos reemplazado nulos

df_copia2.to_csv("datos/attacks_pandas_sklearn.csv")

In [55]:
holi = "😊"